In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [2]:
# read in the path-link incidence file 
# create path-link incidence matrix A

OD_path_dict = {}
with open('../temp_files/path-link_incidence_ext_.txt', 'r') as the_file_:
    for row in the_file_:
        if (':' in row):
            path_list_0 = row.split(': ')[1].split(',')

            path_list_1 = []  # deal with '\n'
            for path in path_list_0:
                if ('\n' in path):
                    path = path.split('\n')[0]
                path_list_1.append((path.split('->')))
            key = row.split(': ')[0]
            value = []
            row_len = len(path_list_1)
            for idx in range(row_len):
                value.append(path_list_1[idx])
            my_list = value
            new_list = []
            for i in my_list:
                if i not in new_list:  # remove duplicate items from a my_list
                    new_list.append(i)
            OD_path_dict[key] = new_list

In [3]:
OD_path_dict['O-D pair (1, 10)']

[['']]

In [4]:
link_label_dict = zload('../temp_files/link_label_dict_ext.pkz')
link_label_dict_ = zload('../temp_files/link_label_dict_ext_.pkz')
link_length_dict = zload('../temp_files/link_length_dict_ext.pkz')

In [5]:
link_length_dict['0'].length

11.385941974850725

In [6]:
OD_pair_label_dict = zload('../temp_files/OD_pair_label_dict_ext.pkz')

In [7]:
'O-D pair ' + OD_pair_label_dict.keys()[0]

'O-D pair (7, 12)'

In [8]:
OD_path_dict['O-D pair (1, 10)']

[['']]

In [9]:
OD_links_dict = {}
for key_ in OD_pair_label_dict.keys():
    key = 'O-D pair ' + key_
    
    path_link_list = []
    for a in OD_path_dict[key]:
    # a = OD_path_dict['O-D pair (1, 10)'][0]

        links_a = []
        for idx in range(len(a))[1:]:
            links_a.append(str(a[idx-1]) + '->' + str(a[idx]))

        # print(links_a)

        link_label_vector = []
        for link in links_a:
            link_label_vector.append(link_label_dict_[link])

        import itertools

        index_list_ = []
        for i in range(len(link_label_vector)):
            L = len(link_label_vector[i])
            index_list_.append(range(L))

        index_list = list(itertools.product(*index_list_))

        for idxes in index_list:
            path_link_list.append([link_label_vector[i][idx] for i, idx in zip(range(len(link_label_vector)), idxes)])
        
        OD_links_dict[key] = path_link_list

zdump(OD_links_dict, '../temp_files/OD_links_dict_ext.pkz')

In [10]:
OD_links_dict['O-D pair (1, 2)'], \
OD_links_dict['O-D pair (1, 3)'], \
OD_links_dict['O-D pair (1, 4)']

([[0]], [[2], [0, 4]], [[0, 6], [2, 5, 6], [2, 8, 11]])

In [11]:
##### add label to routes

i = 0 
routes_dict = {}
routes_dict_ = {}

for j in range(18)[1:]:
    for k in range(18)[1:]:
        if j != k:
            key_ = str((j, k))
            key = 'O-D pair ' + key_
            for route in OD_links_dict[key]:
                routes_dict[str(i)] = route
                routes_dict_[str(route)] = i
                i += 1
num_routes = i
                
zdump(routes_dict, '../temp_files/routes_dict_ext.pkz')
zdump(routes_dict_, '../temp_files/routes_dict__ext.pkz')

In [12]:
routes_dict['5'], routes_dict_[str([2, 8, 11])]

([2, 8, 11], 5)

In [13]:
##### create route labels for each OD pair

ODpair_routes = {}

for j in range(18)[1:]:
    for k in range(18)[1:]:
        if j != k:
            key_ = str((j, k))
            key = 'O-D pair ' + key_
            value = []
            for route in OD_links_dict[key]:
                value.append(routes_dict_[str(route)])
            ODpair_routes[key] = value
            
zdump(ODpair_routes, '../temp_files/ODpair_routes_ext.pkz')

In [14]:
ODpair_routes['O-D pair (1, 5)']

[6, 7, 8]

In [15]:
##### create route-length dict

assert(num_routes) == len(routes_dict)

route_length_dict = {}
for i in range(num_routes):
    key = str(i)
    value = sum([link_length_dict[str(link)].length for link in routes_dict[str(i)]])
    route_length_dict[key] = value
    
zdump(route_length_dict, '../temp_files/route_length_dict_ext.pkz')

In [16]:
route_length_dict['10']

30.553614588975385

In [17]:
import numpy as np

# number of links
num_links = 64
# number of routes (obtained by counting the rows with '->' in 'path-link_incidence.txt')
num_routes = len(routes_dict)

# initialize the path-link incidence matrix
A = np.zeros((num_links, num_routes))

##### create path-link incidence matrix A
# path counts
j = 0  
for r in range(num_routes):
    for m in range(num_links):
        if m in routes_dict[str(r)]:
            A[m, r] = 1
    j += 1
assert(j == num_routes)

zdump(A, '../temp_files/path-link_incidence_matrix_ext.pkz')

In [18]:
np.size(A, 0), np.size(A, 1), A[12, 5]

(64, 1499, 0.0)

In [19]:
ODpair_routes['O-D pair (1, 5)']

[6, 7, 8]

In [20]:
OD_pair_label_dict_ = zload('../temp_files/OD_pair_label_dict__ext.pkz')
OD_pair_label_dict = zload('../temp_files/OD_pair_label_dict_ext.pkz')

In [21]:
OD_pair_label_dict_['0'], OD_pair_label_dict['(1, 2)']

((1, 2), 0)

In [22]:
# calculate route choice probability matrix P

from math import exp

# number of O-D pairs
num_ODpairs = 17 * (17 - 1)


# logit choice parameter
theta = 0.5

P = np.zeros((num_ODpairs, num_routes))
for i in range(num_ODpairs):
    key = 'O-D pair ' + str(OD_pair_label_dict_[str(i)])
    for r in ODpair_routes[key]:
        P[i, r] = exp(- theta * route_length_dict[str(r)]) / \
                    sum([exp(- theta * route_length_dict[str(j)]) \
                         for j in ODpair_routes[key]])
zdump(P, '../temp_files/logit_route_choice_probability_matrix_ext.pkz')

In [23]:
np.size(P, 0), np.size(P, 1)

(272, 1499)

In [24]:
ODpair_routes['O-D pair (1, 5)'], OD_pair_label_dict['(1, 5)']

([6, 7, 8], 3)

In [25]:
P[3, 6], P[3, 7], P[3, 8], route_length_dict[str(6)], route_length_dict[str(7)], route_length_dict[str(8)]

(0.99815583185391132,
 0.0011575928958337549,
 0.00068657525025493226,
 21.864729052807235,
 35.38386235066037,
 36.428626760188095)

In [26]:
np.linalg.matrix_rank(A), np.linalg.matrix_rank(P)

(64, 258)

In [27]:
A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
print(matrix_rank(P))
print("sizes of P are: ")
print(np.size(P, 0))
print(np.size(P, 1))
print("rank of A is: ")
print(matrix_rank(A))
print("sizes of A are: ")
print(np.size(A, 0))
print(np.size(A, 1))
print("rank of PA_t is: ")
print(matrix_rank(PA_t))

258
sizes of P are: 
272
1499
rank of A is: 
64
sizes of A are: 
64
1499
rank of PA_t is: 
54
